# Data Acquisition with Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd

First make the request. The response is a bunch of html.

In [2]:
response = requests.get('https://web-scraping-demo.zgulde.net/news')
html = response.text
html

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>News Example Page</title>\n    <link href="https://unpkg.com/tailwindcss@^2/dist/tailwind.min.css" rel="stylesheet" />\n    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.4.1/font/bootstrap-icons.css" />\n</head>\n<body class="mx-auto max-w-screen-lg pb-32">\n    \n<h1 class="my-5 text-4xl text-center">News!</h1>\n<div class="my-5 text-red-800 px-5 py-3 bg-red-100 font-bold">\n    <p>\n        <i class="bi bi-exclamation-circle text-xl"></i>\n        All data on this page is strictly for demonstration purposes and fake.\n    </p>\n</div>\n<div class="grid gap-y-12">\n    \n    <div class="grid grid-cols-4 gap-x-4 border rounded pr-3 bg-green-50 hover:shadow-lg transition duration-500">\n        <img src="/static/placeholder.png" />\n        <d

We can make more sense of that html with the beautiful soup library.

In [3]:
soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>News Example Page</title>
<link href="https://unpkg.com/tailwindcss@^2/dist/tailwind.min.css" rel="stylesheet"/>
<link href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.4.1/font/bootstrap-icons.css" rel="stylesheet"/>
</head>
<body class="mx-auto max-w-screen-lg pb-32">
<h1 class="my-5 text-4xl text-center">News!</h1>
<div class="my-5 text-red-800 px-5 py-3 bg-red-100 font-bold">
<p>
<i class="bi bi-exclamation-circle text-xl"></i>
        All data on this page is strictly for demonstration purposes and fake.
    </p>
</div>
<div class="grid gap-y-12">
<div class="grid grid-cols-4 gap-x-4 border rounded pr-3 bg-green-50 hover:shadow-lg transition duration-500">
<img src="/static/placeholder.png"/>
<div class="col-span-3 space-y-3 py-3">
<h2 class="text-2xl text-green-900">part population 

From here we can switch between the browser and python and try out different ways of getting different parts of the html document.

We can leverage Google Chrome's developer tools by right clicking and choosing "Inspect". We can then use this html document inspector to help us with our web scraping.

In [4]:
articles = soup.select('.grid.grid-cols-4.gap-x-4.border')
articles[0].select('.italic')[0].select('p')

[<p> 2009-10-02 </p>, <p class="text-right">By Judith Carr </p>]

Bringing it all together:

In [5]:
def process_article(article):
    date, author = articles[0].select('.italic')[0].select('p')
    return {
        'title': article.h2.text,
        'date': date.text,
        'author': author.text
    }

pd.DataFrame([process_article(article) for article in articles])

,title,date,author
0,part population head,2009-10-02,By Judith Carr
1,say lawyer again,2009-10-02,By Judith Carr
2,less administration finally,2009-10-02,By Judith Carr
3,idea much example,2009-10-02,By Judith Carr
4,interesting could real,2009-10-02,By Judith Carr
5,treat system stop,2009-10-02,By Judith Carr
6,believe after physical,2009-10-02,By Judith Carr
7,financial ground remember,2009-10-02,By Judith Carr
8,million event practice,2009-10-02,By Judith Carr
9,loss tonight democratic,2009-10-02,By Judith Carr


# Web Scraping for People

In [6]:
response = requests.get('https://web-scraping-demo.zgulde.net/people', headers={'user-agent': 'Codeup DS Germain'})
soup = BeautifulSoup(response.text)

In [7]:
people = soup.select('.person')

In [8]:
person = people[0]

In [9]:
# putting it all together
import re


def parse_person(person):
    name = person.h2.text
    # .p finds the first p element; or element with a tag name of "p"
    quote = person.p.text.strip()
    # email
    email = person.select('.email')[0].text
    # phone
    phone = person.select('.phone')[0].text
    # address
    address = person.select('.address')[0].text.strip()
    address = re.sub(r'\s{2,}', ' ', address)
    
    return {'name': name, 'quote': quote, 'email': email, 'phone': phone, 'address': address}

In [10]:
pd.DataFrame([parse_person(person) for person in people])

,name,quote,email,phone,address
0,Timothy Rodriguez,"""Implemented responsive collaboration""",mark88@butler.org,5839171255,"340 Garcia Freeway East Wendy, WI 71650"
1,Nicholas Hernandez,"""Universal national Graphical User Interface""",mhenson@hotmail.com,911.637.7841,"476 Jennifer Dam Suite 111 Jenniferfort, WI 90023"
2,Edwin Dixon,"""Synergized value-added secured line""",wadesharon@hotmail.com,(583)418-3031,"62947 William Mission Apt. 602 New Jessica, WI..."
3,Jose Mcclure,"""Function-based full-range forecast""",hayesronald@yahoo.com,916.295.2583,"8191 Sarah Point East Elizabeth, IL 12068"
4,Angela Golden,"""Sharable grid-enabled knowledge user""",charris@gmail.com,785-041-7348x5994,"348 Jennifer Flats Pattonstad, ND 11761"
5,Kristen Silva,"""Face-to-face clear-thinking interface""",andreasilva@vargas-owens.com,(190)599-5854,"08554 Lawrence Rapids East Julia, MD 42852"
6,Philip Fields,"""Persevering analyzing intranet""",mhuffman@davis-ford.net,(513)818-7492,"535 David Club West Kyleland, SC 63892"
7,Alan Hicks,"""Phased exuding alliance""",josephmanning@foster-alexander.com,+1-113-389-6231x17375,"2714 Wendy Overpass Port Christine, NV 12276"
8,Robin Jackson,"""Extended responsive artificial intelligence""",jonathan52@nielsen.com,001-389-161-6925x05185,"636 Oconnell Extension New Erin, NY 07837"
9,Eric Phillips,"""Object-based analyzing hardware""",riveratravis@smith.com,264.619.7899,"5268 Adam Park Lake Benjamin, NE 39068"



Web Scraping Etiquitte
respect the robots.txt file if present

Wikipedia: Robots exclusion standard
robotstxt.org
codeup's robots.txt
use a descriptive user agent

requests.get('http://example.com', headers={'user-agent': 'codeup data science germain cohort'})